https://auto.gluon.ai/stable/tutorials/timeseries/forecasting-chronos.html#incorporating-the-covariates

The predictions of the covariate regressor are subtracted from the target column, and the univariate model then forecasts the residuals.



In [1]:
import datetime

import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from sklearn.metrics import root_mean_squared_error

In [2]:
# Experiment設定
EXPERIMENT_NAME = 'ChronosBolt_0shot_covariate'
MODEL = "bolt_base"
FORECAST_HORIZON = 6

In [3]:
df_train = pd.read_csv('../data/walmart_ts_6_fcst_grp_train.csv')
df_test = pd.read_csv('../data/walmart_ts_6_fcst_grp_test.csv')
print(df_train.shape, df_test.shape)
#df_train.head()

# 日付カラムを型変換する場合
#df_train['Date'] = pd.to_datetime(df_train['Date'])
#df_test['Date'] = pd.to_datetime(df_test['Date'])

# 単一のグループIDを作成
df_train['Store_Dept'] = [str(i)+'_'+str(j) for i,j in zip(df_train['Store'], df_train['Dept'])]
df_test['Store_Dept'] = [str(i)+'_'+str(j) for i,j in zip(df_test['Store'], df_test['Dept'])]

# 利用カラムのみを選択
df_train = df_train[['Store_Dept','Date','Weekly_Sales', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'IsHoliday']]
df_test = df_test[['Store_Dept','Date','Weekly_Sales', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'IsHoliday']]
print(df_train.shape, df_test.shape)
df_train.head()

(702, 11) (36, 11)
(702, 9) (36, 9)


,Store_Dept,Date,Weekly_Sales,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,IsHoliday
0,4_4,2010/2/5,59554.57,-1.0,-1.0,-1.0,-1.0,-1.0,0
1,4_4,2010/2/12,54069.82,-1.0,-1.0,-1.0,-1.0,-1.0,1
2,4_4,2010/2/19,53939.17,-1.0,-1.0,-1.0,-1.0,-1.0,0
3,4_4,2010/2/26,54687.08,-1.0,-1.0,-1.0,-1.0,-1.0,0
4,4_4,2010/3/5,56959.02,-1.0,-1.0,-1.0,-1.0,-1.0,0


In [4]:
# TimeSeriesDataFrameへ変換
train_data = TimeSeriesDataFrame.from_data_frame(
    df_train,
    id_column='Store_Dept',
    timestamp_column="Date"
)
print(type(train_data))
train_data.head()

<class 'autogluon.timeseries.dataset.ts_dataframe.TimeSeriesDataFrame'>


Weekly_Sales  MarkDown1  MarkDown2  MarkDown3  MarkDown4  \
item_id timestamp                                                              
4_4     2010-02-05      59554.57       -1.0       -1.0       -1.0       -1.0   
        2010-02-12      54069.82       -1.0       -1.0       -1.0       -1.0   
        2010-02-19      53939.17       -1.0       -1.0       -1.0       -1.0   
        2010-02-26      54687.08       -1.0       -1.0       -1.0       -1.0   
        2010-03-05      56959.02       -1.0       -1.0       -1.0       -1.0   

                    MarkDown5  IsHoliday  
item_id timestamp                         
4_4     2010-02-05       -1.0          0  
        2010-02-12       -1.0          1  
        2010-02-19       -1.0          0  
        2010-02-26       -1.0          0  
        2010-03-05       -1.0          0

In [5]:
test_data = TimeSeriesDataFrame.from_data_frame(
    df_test,
    #id_column=['Store', 'Dept'],
    id_column='Store_Dept',
    timestamp_column="Date"
)
test_data.head()

Weekly_Sales  MarkDown1  MarkDown2  MarkDown3  MarkDown4  \
item_id timestamp                                                              
4_4     2012-05-04      60576.41   13737.91       -1.0      93.65    6993.97   
        2012-05-11      57583.94   20499.88       86.0     103.05    5005.96   
        2012-05-18      60832.91    6531.12       -1.0      99.91    2290.18   
        2012-05-25      57781.87    4330.02      436.0     862.31    3268.41   
        2012-06-01      62758.52   10165.22        8.6      47.93    2676.33   

                    MarkDown5  IsHoliday  
item_id timestamp                         
4_4     2012-05-04    4541.89          0  
        2012-05-11    3815.16          0  
        2012-05-18    2521.84          0  
        2012-05-25    3749.53          0  
        2012-06-01    4896.34          0

In [8]:
predictor = TimeSeriesPredictor(
    prediction_length=FORECAST_HORIZON,
    target="Weekly_Sales",
    known_covariates_names=['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'IsHoliday'],
).fit(
    train_data,
    hyperparameters={
        "Chronos": [
            # Zero-shot model WITHOUT covariates
            {
                "model_path": "bolt_small",
                "ag_args": {"name_suffix": "ZeroShot"},
            },
            # Chronos-Bolt (Small) combined with CatBoost on covariates
            {
                "model_path": "bolt_small",
                "covariate_regressor": "CAT",
                "target_scaler": "standard",
                "ag_args": {"name_suffix": "WithRegressor"},
            },
        ],
    },
    enable_ensemble=False,
    time_limit=60,
)

Beginning AutoGluon training... Time limit = 60s
AutoGluon will save models to '/Users/YShimada-MBP16/Documents/Code_Github/DL_for_ImageData_and_Finetuning/TimeSeries_WM_simple/code/AutogluonModels/ag-20250228_055457'
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.14
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 24.3.0: Thu Jan  2 20:22:00 PST 2025; root:xnu-11215.81.4~3/RELEASE_X86_64
CPU Count:          16
GPU Count:          0
Memory Avail:       14.32 GB / 32.00 GB (44.8%)
Disk Space Avail:   1433.46 GB / 1863.39 GB (76.9%)

Fitting with arguments:
{'enable_ensemble': False,
 'eval_metric': WQL,
 'hyperparameters': {'Chronos': [{'ag_args': {'name_suffix': 'ZeroShot'},
                                  'model_path': 'bolt_small'},
                                 {'ag_args': {'name_suffix': 'WithRegressor'},
                                  'covariate_regressor': 'CAT',
         

In [12]:
predictor.model_names()

['ChronosZeroShot[bolt_small]', 'ChronosWithRegressor[bolt_small]']

In [14]:
predictor.leaderboard(train_data)

Additional data provided, testing on additional data. Resulting leaderboard will be sorted according to test score (`score_test`).


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time_marginal,fit_order
0,ChronosWithRegressor[bolt_small],-0.033255,-0.033255,1.072695,1.006809,2.649229,2
1,ChronosZeroShot[bolt_small],-0.035596,-0.035596,1.188568,1.478391,4.070458,1


In [19]:
predictor.predict(train_data, known_covariates=test_data)

Model not specified in predict, will default to the model with the best validation score: ChronosWithRegressor[bolt_small]


mean           0.1           0.2           0.3  \
item_id timestamp                                                            
4_4     2012-05-04  57491.403139  54222.848723  55230.357360  56030.077945   
        2012-05-11  57509.401190  54167.476679  55188.990673  56001.909467   
        2012-05-18  55984.466534  52616.551695  53646.245062  54470.128956   
        2012-05-25  61883.195367  58485.224094  59552.735678  60371.336170   
        2012-06-01  57250.852410  53851.109956  54921.445141  55750.607825   
        2012-06-08  58776.708518  55380.328737  56462.463486  57287.758271   
8_13    2012-05-04  24046.211716  22581.393880  23056.592754  23416.871442   
        2012-05-11  24325.578075  22771.370379  23264.669280  23649.588136   
        2012-05-18  24831.852307  23250.857501  23754.341518  24144.856651   
        2012-05-25  25283.201642  23688.794472  24204.271655  24594.035570   
        2012-06-01  24094.569671  22509.670855  23022.031433  23411.142391   
        2012-06-08  25252.827855  23667.935752  24181.904863  24572.698446   
1_13    2012-05-04  40932.594290  38918.087362  39581.916155  40054.656163   
        2012-05-11  40556.059505  38357.269049  39082.069812  39617.365148   
        2012-05-18  39841.872976  37479.989892  38265.272755  38839.381856   
        2012-05-25  41403.264180  38793.114858  39651.237472  40279.334211   
        2012-06-01  40497.770957  37739.852636  38667.167999  39332.889133   
        2012-06-08  42047.373599  39081.013205  40071.936598  40783.805851   
13_8    2012-05-04  34826.111262  32831.259625  33487.475240  33979.120163   
        2012-05-11  34572.934390  32525.359709  33196.178886  33702.378892   
        2012-05-18  34051.954681  31992.496210  32671.238107  33179.586968   
        2012-05-25  35075.371582  33002.710864  33696.242700  34201.479720   
        2012-06-01  34680.861722  32617.617937  33309.571642  33815.027679   
        2012-06-08  35851.236615  33786.194086  34482.554624  34991.099327   
19_8    2012-05-04  43132.679573  40834.004246  41608.452938  42173.490996   
        2012-05-11  41564.868582  39151.555829  39958.415371  40553.935288   
        2012-05-18  41733.824871  39257.816241  40082.333324  40696.016505   
        2012-05-25  44454.533999  41946.615925  42796.316252  43409.785918   
        2012-06-01  40865.171099  38371.801652  39218.884756  39833.018320   
        2012-06-08  42680.608327  40167.870260  41031.153982  41654.260261   
31_13   2012-05-04  43282.465845  41041.684998  41789.688747  42351.183854   
        2012-05-11  43081.079756  40623.556008  41424.623450  42041.060231   
        2012-05-18  44147.978449  41596.154587  42420.684769  43061.939784   
        2012-05-25  44143.226791  41547.077854  42395.630033  43044.570116   
        2012-06-01  42700.539755  40094.236050  40949.168652  41603.055447   
        2012-06-08  43451.860356  40808.462811  41676.036316  42340.932653   

                             0.4           0.5           0.6           0.7  \
item_id timestamp                                                            
4_4     2012-05-04  56772.898075  57491.403139  58254.277402  59115.951709   
        2012-05-11  56762.576704  57509.401190  58301.842577  59208.457767   
        2012-05-18  55235.069209  55984.466534  56785.208777  57702.725857   
        2012-05-25  61133.419430  61883.195367  62680.578055  63611.634569   
        2012-06-01  56511.045414  57250.852410  58061.446463  59015.533020   
        2012-06-08  58041.467265  58776.708518  59585.228136  60537.039909   
8_13    2012-05-04  23739.147047  24046.211716  24364.729557  24725.791425   
        2012-05-11  23994.649285  24325.578075  24667.797855  25059.600061   
        2012-05-18  24496.488186  24831.852307  25182.537202  25581.353288   
        2012-05-25  24946.662257  25283.201642  25634.573999  26035.646116   
        2012-06-01  23761.716171  24094.569671  24448.794106  24859.151059   
        2012-06-08  24920.096277  25252.827855  2

In [10]:
%%time

log_dir = EXPERIMENT_NAME +'_'+ datetime.datetime.now().strftime('%Y-%m-%d_%Hh%Mm%Ss')
print(log_dir)

predictor = TimeSeriesPredictor(
    target="Weekly_Sales",
    eval_metric="RMSE",
    prediction_length=FORECAST_HORIZON,
    path="./tmp/"+log_dir,
    
    known_covariates_names=['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'IsHoliday'],
).fit(
    train_data,
    hyperparameters={
        "Chronos": [
            {
                "model_path": MODEL,
                #"model_path": "bolt_small",
                #"covariate_regressor": "XGB",  # XGBoost  -> カーネルが落ちてしまう、、。
                "covariate_regressor": "CAT",  # CatBoost
                "target_scaler": "standard",
                "ag_args": {"name_suffix": "WithRegressor"},
            },
        ],
    },
    enable_ensemble=False,
    time_limit=60,
)

Beginning AutoGluon training... Time limit = 60s
AutoGluon will save models to '/Users/YShimada-MBP16/Documents/Code_Github/DL_for_ImageData_and_Finetuning/TimeSeries_WM_simple/code/tmp/ChronosBolt_0shot_covariate_2025-02-28_15h00m59s'
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.14
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 24.3.0: Thu Jan  2 20:22:00 PST 2025; root:xnu-11215.81.4~3/RELEASE_X86_64
CPU Count:          16
GPU Count:          0
Memory Avail:       14.21 GB / 32.00 GB (44.4%)
Disk Space Avail:   1433.49 GB / 1863.39 GB (76.9%)

Fitting with arguments:
{'enable_ensemble': False,
 'eval_metric': RMSE,
 'hyperparameters': {'Chronos': [{'ag_args': {'name_suffix': 'WithRegressor'},
                                  'covariate_regressor': 'CAT',
                                  'model_path': 'bolt_small',
                                  'target_scaler': 'standard'}]},

ChronosBolt_0shot_covariate_2025-02-28_15h00m59s


	-2425.8846    = Validation score (-RMSE)
	1.04    s     = Training runtime
	1.19    s     = Validation (prediction) runtime
Training complete. Models trained: ['ChronosWithRegressor[bolt_small]']
Total runtime: 2.24 s
Best model: ChronosWithRegressor[bolt_small]
Best model score: -2425.8846


CPU times: user 4.05 s, sys: 860 ms, total: 4.91 s
Wall time: 2.37 s


In [11]:
predictor.predict(train_data, known_covariates=test_data)

Model not specified in predict, will default to the model with the best validation score: ChronosWithRegressor[bolt_small]


mean           0.1           0.2           0.3  \
item_id timestamp                                                            
4_4     2012-05-04  57491.403139  54222.848723  55230.357360  56030.077945   
        2012-05-11  57509.401190  54167.476679  55188.990673  56001.909467   
        2012-05-18  55984.466534  52616.551695  53646.245062  54470.128956   
        2012-05-25  61883.195367  58485.224094  59552.735678  60371.336170   
        2012-06-01  57250.852410  53851.109956  54921.445141  55750.607825   
        2012-06-08  58776.708518  55380.328737  56462.463486  57287.758271   
8_13    2012-05-04  24046.211716  22581.393880  23056.592754  23416.871442   
        2012-05-11  24325.578075  22771.370379  23264.669280  23649.588136   
        2012-05-18  24831.852307  23250.857501  23754.341518  24144.856651   
        2012-05-25  25283.201642  23688.794472  24204.271655  24594.035570   
        2012-06-01  24094.569671  22509.670855  23022.031433  23411.142391   
        2012-06-08  25252.827855  23667.935752  24181.904863  24572.698446   
1_13    2012-05-04  40932.594290  38918.087362  39581.916155  40054.656163   
        2012-05-11  40556.059505  38357.269049  39082.069812  39617.365148   
        2012-05-18  39841.872976  37479.989892  38265.272755  38839.381856   
        2012-05-25  41403.264180  38793.114858  39651.237472  40279.334211   
        2012-06-01  40497.770957  37739.852636  38667.167999  39332.889133   
        2012-06-08  42047.373599  39081.013205  40071.936598  40783.805851   
13_8    2012-05-04  34826.111262  32831.259625  33487.475240  33979.120163   
        2012-05-11  34572.934390  32525.359709  33196.178886  33702.378892   
        2012-05-18  34051.954681  31992.496210  32671.238107  33179.586968   
        2012-05-25  35075.371582  33002.710864  33696.242700  34201.479720   
        2012-06-01  34680.861722  32617.617937  33309.571642  33815.027679   
        2012-06-08  35851.236615  33786.194086  34482.554624  34991.099327   
19_8    2012-05-04  43132.679573  40834.004246  41608.452938  42173.490996   
        2012-05-11  41564.868582  39151.555829  39958.415371  40553.935288   
        2012-05-18  41733.824871  39257.816241  40082.333324  40696.016505   
        2012-05-25  44454.533999  41946.615925  42796.316252  43409.785918   
        2012-06-01  40865.171099  38371.801652  39218.884756  39833.018320   
        2012-06-08  42680.608327  40167.870260  41031.153982  41654.260261   
31_13   2012-05-04  43282.465845  41041.684998  41789.688747  42351.183854   
        2012-05-11  43081.079756  40623.556008  41424.623450  42041.060231   
        2012-05-18  44147.978449  41596.154587  42420.684769  43061.939784   
        2012-05-25  44143.226791  41547.077854  42395.630033  43044.570116   
        2012-06-01  42700.539755  40094.236050  40949.168652  41603.055447   
        2012-06-08  43451.860356  40808.462811  41676.036316  42340.932653   

                             0.4           0.5           0.6           0.7  \
item_id timestamp                                                            
4_4     2012-05-04  56772.898075  57491.403139  58254.277402  59115.951709   
        2012-05-11  56762.576704  57509.401190  58301.842577  59208.457767   
        2012-05-18  55235.069209  55984.466534  56785.208777  57702.725857   
        2012-05-25  61133.419430  61883.195367  62680.578055  63611.634569   
        2012-06-01  56511.045414  57250.852410  58061.446463  59015.533020   
        2012-06-08  58041.467265  58776.708518  59585.228136  60537.039909   
8_13    2012-05-04  23739.147047  24046.211716  24364.729557  24725.791425   
        2012-05-11  23994.649285  24325.578075  24667.797855  25059.600061   
        2012-05-18  24496.488186  24831.852307  25182.537202  25581.353288   
        2012-05-25  24946.662257  25283.201642  25634.573999  26035.646116   
        2012-06-01  23761.716171  24094.569671  24448.794106  24859.151059   
        2012-06-08  24920.096277  25252.827855  2

## Finetune

In [12]:
%%time

log_dir = EXPERIMENT_NAME +'_'+ datetime.datetime.now().strftime('%Y-%m-%d_%Hh%Mm%Ss')
print(log_dir)

predictor = TimeSeriesPredictor(
    target="Weekly_Sales",
    eval_metric="RMSE",
    prediction_length=FORECAST_HORIZON,
    path="./tmp/"+log_dir,
    
    known_covariates_names=['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'IsHoliday'],
).fit(
    train_data,
    hyperparameters={
        "Chronos": [
            {
                "model_path": MODEL,
                #"model_path": "bolt_small",
                #"covariate_regressor": "XGB",  # XGBoost  -> カーネルが落ちてしまう、、。
                "covariate_regressor": "CAT",  # CatBoost
                "target_scaler": "standard",
                "fine_tune": True, 
                "ag_args": {"name_suffix": "WithRegressorFineTune"},
            },
        ],
    },
    enable_ensemble=False,
    time_limit=60,
)

Beginning AutoGluon training... Time limit = 60s
AutoGluon will save models to '/Users/YShimada-MBP16/Documents/Code_Github/DL_for_ImageData_and_Finetuning/TimeSeries_WM_simple/code/tmp/ChronosBolt_0shot_covariate_2025-02-28_15h04m24s'
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.14
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 24.3.0: Thu Jan  2 20:22:00 PST 2025; root:xnu-11215.81.4~3/RELEASE_X86_64
CPU Count:          16
GPU Count:          0
Memory Avail:       14.15 GB / 32.00 GB (44.2%)
Disk Space Avail:   1433.50 GB / 1863.39 GB (76.9%)

Fitting with arguments:
{'enable_ensemble': False,
 'eval_metric': RMSE,
 'hyperparameters': {'Chronos': [{'ag_args': {'name_suffix': 'WithRegressorFineTune'},
                                  'covariate_regressor': 'CAT',
                                  'fine_tune': True,
                                  'model_path': 'bolt_base',
     

ChronosBolt_0shot_covariate_2025-02-28_15h04m24s


	Fine-tuning on the CPU detected. We recommend using a GPU for faster fine-tuning of Chronos.
	Saving fine-tuned model to /Users/YShimada-MBP16/Documents/Code_Github/DL_for_ImageData_and_Finetuning/TimeSeries_WM_simple/code/tmp/ChronosBolt_0shot_covariate_2025-02-28_15h04m24s/models/ChronosWithRegressorFineTune[bolt_base]/W0/fine-tuned-ckpt
	-2427.7808    = Validation score (-RMSE)
	51.03   s     = Training runtime
	0.19    s     = Validation (prediction) runtime
Training complete. Models trained: ['ChronosWithRegressorFineTune[bolt_base]']
Total runtime: 51.24 s
Best model: ChronosWithRegressorFineTune[bolt_base]
Best model score: -2427.7808


CPU times: user 5min 26s, sys: 26.7 s, total: 5min 53s
Wall time: 51.4 s


In [13]:
predictor.predict(train_data, known_covariates=test_data)

Model not specified in predict, will default to the model with the best validation score: ChronosWithRegressorFineTune[bolt_base]


mean           0.1           0.2           0.3  \
item_id timestamp                                                            
4_4     2012-05-04  57799.759006  54978.919943  55850.640075  56498.387089   
        2012-05-11  58189.506089  55299.605612  56164.171796  56866.927217   
        2012-05-18  56828.868308  53825.784341  54743.393670  55459.566847   
        2012-05-25  62825.036965  59805.663373  60733.501304  61464.950600   
        2012-06-01  58548.645713  55344.821424  56348.331454  57106.240834   
        2012-06-08  59600.066982  56509.092333  57499.243010  58247.983158   
8_13    2012-05-04  23973.659727  22594.983428  23060.921855  23393.154007   
        2012-05-11  24264.628762  22818.058432  23295.900657  23655.420731   
        2012-05-18  24777.288393  23281.016464  23784.124696  24152.220562   
        2012-05-25  25212.369313  23723.495787  24226.504702  24595.708818   
        2012-06-01  24070.222260  22560.019556  23069.871847  23441.116807   
        2012-06-08  25184.672104  23688.736009  24204.708985  24575.019192   
1_13    2012-05-04  40964.332140  38821.314208  39580.349322  40074.948059   
        2012-05-11  40415.041601  38213.759743  38971.522870  39518.908016   
        2012-05-18  39598.048726  37406.837830  38172.261072  38700.422933   
        2012-05-25  41114.719808  38882.245685  39641.822299  40182.040997   
        2012-06-01  40255.053788  37706.212429  38570.470193  39171.023405   
        2012-06-08  41910.954670  39277.651817  40221.126218  40848.200769   
13_8    2012-05-04  34840.591215  33159.129003  33719.833945  34130.191079   
        2012-05-11  34634.770849  32913.803776  33491.874348  33920.456577   
        2012-05-18  34103.746991  32329.530380  32944.782748  33386.427103   
        2012-05-25  35256.536221  33459.468085  34084.675179  34527.481453   
        2012-06-01  35146.525219  33282.910853  33930.052259  34391.331230   
        2012-06-08  36383.033895  34484.600343  35164.311600  35633.056760   
19_8    2012-05-04  42923.204226  40704.096698  41452.336080  41990.735638   
        2012-05-11  41386.311397  39094.376302  39852.704445  40426.221956   
        2012-05-18  41499.577267  39174.787317  39961.009731  40535.574798   
        2012-05-25  44353.302366  42032.006462  42817.337040  43389.761859   
        2012-06-01  40874.401878  38525.076882  39320.724553  39893.278501   
        2012-06-08  42636.881496  40267.866530  41083.099653  41671.801232   
31_13   2012-05-04  42560.591434  40624.057786  41296.014155  41759.402750   
        2012-05-11  42827.752960  40696.517467  41413.409629  41947.146395   
        2012-05-18  44415.569549  42126.626749  42922.494745  43480.882200   
        2012-05-25  44221.865008  41999.426905  42776.995966  43330.413609   
        2012-06-01  42567.587497  40326.445249  41121.694052  41674.775650   
        2012-06-08  43294.058632  41014.266222  41840.215286  42411.269462   

                             0.4           0.5           0.6           0.7  \
item_id timestamp                                                            
4_4     2012-05-04  57119.902140  57799.759006  58517.426033  59345.850140   
        2012-05-11  57493.578658  58189.506089  58933.595066  59787.407673   
        2012-05-18  56153.898045  56828.868308  57546.641746  58380.328597   
        2012-05-25  62143.583022  62825.036965  63552.290098  64418.245281   
        2012-06-01  57828.665054  58548.645713  59305.550293  60199.999781   
        2012-06-08  58909.591340  59600.066982  60376.332691  61272.861031   
8_13    2012-05-04  23684.946812  23973.659727  24265.109852  24598.641448   
        2012-05-11  23961.297908  24264.628762  24578.567052  24936.703201   
        2012-05-18  24477.200139  24777.288393  25087.673716  25441.566917   
        2012-05-25  24914.763130  25212.369313  25523.511951  25888.885972   
        2012-06-01  23762.637144  24070.222260  24383.722206  24755.051163   
        2012-06-08  24886.699116  25184.672104  2